# Import Packages

In [124]:
# declare imports
# import glob
# import pandas as pd
# import dask.dataframe as dd
# from pandas.api.types import is_numeric_dtype
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# from pathlib import Path
# import numpy as np
# import seaborn as sns
# import colorcet as cc
# import math
# from pearllib import group_title_dict, NA_ACCORD_group_title_dict
# import yaml

In [4]:
import argparse
from datetime import datetime
from pathlib import Path
import os
import sys
import dask.dataframe as dd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml
import colorcet as cc

path_to_functions = os.path.abspath("../src")

# Add this folder to the system path
sys.path.append(path_to_functions)

from pearl.post_processing.bmi import (
    add_summary,
    calc_overall_risk,
    calc_percentage,
    calc_percentage_and_add_summary,
    calc_risk_by_group,
    clean_control,
    create_summary_table,
    group_order,
    group_order_with_sub_total,
    group_title_dict,
    palette,
    rearrange_group_order,
    round_thousand,
    calc_dm_prop,
    add_sub_total,
    calc_overall_bmi_risk,
)

start_time = datetime.now()
df_summary_dict = {}

# Define the argument parser

baseline_dir = Path('../out/S0_10/combined')
variable_dir = Path('../out/S3_10/combined')
out_dir = Path('../results')

start_year = 2013
end_year = 2017

# Define Group Order

In [2]:
group_order = [
    "Black HET Women",
    "White HET Women",
    "Hispanic HET Women",
    "Black HET Men",
    "White HET Men",
    "Hispanic HET Men",
    "Black WWID",
    "White WWID",
    "Hispanic WWID",
    "Black MWID",
    "White MWID",
    "Hispanic MWID",
    "Black MSM",
    "White MSM",
    "Hispanic MSM",
    "Overall"
]

# Plot Color Setting

In [5]:
#define color pallete
palette = sns.color_palette(cc.glasbey_light, n_colors=16)

## calc_percentage

In [6]:
round_thousand = lambda x: int(math.ceil(x / 100.0)) * 100 if  x > 1000 else x

# Old Risk Calculation

In [36]:
def calc_risk_by_group_old(df, years_follow_up):

    # filter for only x-year follow up with dm
    df_follow_up = df.loc[(df['years_after_h1yy'] > 0) & (df['years_after_h1yy'] <= years_follow_up)]

    # group by replication and group and sum
    df_follow_up_sum = df_follow_up.groupby(['group', 'replication'])['n'].sum().reset_index()
    df_follow_up_sum = df_follow_up_sum.rename(columns={'n': 'dm_num'})

    # group by replication and group and sum
    df_all_sum = df.groupby(['group', 'replication'])['n'].sum().reset_index()
    df_all_sum = df_all_sum.rename(columns={'n': 'num'})

    # merge dataframes
    group_dm_risk_table = dd.merge(df_follow_up_sum, df_all_sum, how='left')

    # calculate risk
    group_dm_risk_table['risk'] = group_dm_risk_table['dm_num'] / group_dm_risk_table['num']

    return group_dm_risk_table

# Define Data Path

In [7]:
config_type = 'SA1_10'

config_file_path = f'../config/S0_{config_type}.yaml' #gitignore

# Load the YAML file
with open(config_file_path, 'r') as file:
    config_data = yaml.safe_load(file)

coverage_rate = config_data['bmi_intervention_coverage']
effectiveness = config_data['bmi_intervention_effectiveness']

if config_type != 'baseline':
    # Set path to input folder
    s0_data_dir = Path(f"../out/S0_{config_type}/combined/") #gitignore
    s1_data_dir = Path(f"../out/S3_{config_type}/combined/") #gitignore
else:
    # For Baseline running
    s0_data_dir = Path(f"../out/S0_10/combined/") #gitignore
    s1_data_dir = Path(f"../out/S3_10/combined/") #gitignore

In [8]:
config_data

{'group_names': ['msm_white_male',
  'msm_black_male',
  'msm_hisp_male',
  'idu_white_male',
  'idu_black_male',
  'idu_hisp_male',
  'idu_white_female',
  'idu_black_female',
  'idu_hisp_female',
  'het_white_male',
  'het_black_male',
  'het_hisp_male',
  'het_white_female',
  'het_black_female',
  'het_hisp_female'],
 'num_cpus': 150,
 'replications': 10,
 'new_dx': 'base',
 'mortality_model': 'by_sex_race_risk',
 'mortality_threshold_flag': 1,
 'final_year': 2035,
 'verbose': 0,
 'idu_threshold': '2x',
 'bmi_intervention': 1,
 'bmi_intervention_scenario': 0,
 'bmi_intervention_start_year': 2013,
 'bmi_intervention_end_year': 2017,
 'bmi_intervention_coverage': 0.75,
 'bmi_intervention_effectiveness': 1.0}

# Create Summary Table

In [9]:
SA_summary_df = pd.DataFrame()
SA_summary_df['group'] = [
        "Black HET Women",
        "White HET Women",
        "Hispanic HET Women",
        "Black HET Men",
        "White HET Men",
        "Hispanic HET Men",
        "Black WWID",
        "White WWID",
        "Hispanic WWID",
        "Black MWID",
        "White MWID",
        "Hispanic MWID",
        "Black MSM",
        "White MSM",
        "Hispanic MSM",
        "Overall"
    ]

# Control Arm

## Number of People received BMI intervention

In [10]:
# we will look at the "bmi_int_dm_prev.h5" for S0
bmi_int_dm_prev = dd.read_parquet(s0_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
control_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=True, only_received=True)

# sum across replications, group, and years_after_h1yy
control_bmi_int_dm_prev_agg = control_bmi_int_dm_prev.groupby(['group', 'years_after_h1yy','replication'])['n'].sum().reset_index().compute()

# display table
control_bmi_int_dm_prev_agg

,group,years_after_h1yy,replication,n
0,het_black_female,-2017,0,731
1,het_black_female,-2017,1,694
2,het_black_female,-2017,2,639
3,het_black_female,-2017,3,691
4,het_black_female,-2017,4,661
...,...,...,...,...
4312,overall,22,5,339
4313,overall,22,6,327
4314,overall,22,7,364
4315,overall,22,8,326


In [11]:
df = control_bmi_int_dm_prev_agg.groupby(['group', 'replication'])[['n']].sum().reset_index()
df['group'] = df['group'].map(group_title_dict)
df = df.groupby('group')[['n']].apply(lambda x: x.quantile([0.025,0.5,0.975])).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.0f} [{:.0f} - {:.0f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Control|Number receiving intervention'] = df['formatted']

In [14]:
# Now we will work on the remaining percentage columns
bmi_int_cascade = dd.read_parquet(s0_data_dir / 'bmi_int_cascade.parquet').reset_index()
# filter for only starting h1yy after 2013 and before 2017
s0_bmi_int_cascade = bmi_int_cascade.loc[(bmi_int_cascade['h1yy'] >= start_year) & (bmi_int_cascade['h1yy'] <= 2017)]


# we will look at the "bmi_int_dm_prev.h5" for S0
bmi_int_dm_prev = dd.read_parquet(s0_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
control_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=False)

# sum across replications, group, and years_after_h1yy
control_bmi_int_dm_prev_agg = control_bmi_int_dm_prev.groupby(['group', 'years_after_h1yy','replication',"time_exposure_to_risk"])['n'].sum().reset_index().compute()

# display table
control_bmi_int_dm_prev_agg

,group,years_after_h1yy,replication,time_exposure_to_risk,n
0,het_black_female,-2017,0,-2017,1249
1,het_black_female,-2017,0,0,18
2,het_black_female,-2017,0,1,13
3,het_black_female,-2017,0,2,17
4,het_black_female,-2017,0,3,18
...,...,...,...,...,...
66067,overall,22,7,22,15
66068,overall,22,8,-2013,534
66069,overall,22,8,22,18
66070,overall,22,9,-2013,555


## number of new dm & risk events during 7 year follow up

In [15]:
group_dm_risk_table = calc_risk_by_group(control_bmi_int_dm_prev_agg, 7).compute()

group_dm_risk_table['group'] = group_dm_risk_table['group'].map(group_title_dict)

# New DM
df = group_dm_risk_table.groupby('group')[['dm_num']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.1f} [{:.1f} - {:.1f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Control|Number of new dm events during 7 year follow up'] = df['formatted']

# DM Risk
df = group_dm_risk_table.groupby('group')[['risk']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.3f} [{:.3f} - {:.3f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Control|7 year risk of dm'] = df['formatted']

## Number of dm per 1000 receiving the intervention

In [38]:
# we will look at the "bmi_int_dm_prev.h5" for S0
bmi_int_dm_prev = dd.read_parquet(s0_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
control_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=True, only_received=True)

# sum across replications, group, and years_after_h1yy
control_bmi_int_dm_prev_agg = control_bmi_int_dm_prev.groupby(['group', 'years_after_h1yy','replication', "time_exposure_to_risk"])['n'].sum().reset_index().compute()

group_dm_risk_table = calc_risk_by_group_old(control_bmi_int_dm_prev_agg, 7).compute()

group_dm_risk_table['group'] = group_dm_risk_table['group'].map(group_title_dict)

group_dm_risk_table['dm_per_1000'] = np.round(group_dm_risk_table['risk']*1000,0)

# New DM
df = group_dm_risk_table.groupby('group')[['dm_per_1000']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.1f} [{:.1f} - {:.1f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Control|nubmer of dm event per 1000 people receiving intervention'] = df['formatted']

# Intervention Arm

## number of people received intervention

In [17]:
# we will look at the "bmi_int_dm_prev.h5" for S1
bmi_int_dm_prev = dd.read_parquet(s1_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
control_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=True, only_received=True)

# sum across replications, group, and years_after_h1yy
control_bmi_int_dm_prev_agg = control_bmi_int_dm_prev.groupby(['group', 'years_after_h1yy','replication', "time_exposure_to_risk"])['n'].sum().reset_index().compute()

# display table
control_bmi_int_dm_prev_agg

df = control_bmi_int_dm_prev_agg.groupby(['group', 'replication'])[['n']].sum().reset_index()
df['group'] = df['group'].map(group_title_dict)
df = df.groupby('group')[['n']].apply(lambda x: x.quantile([0.025,0.5,0.975])).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.0f} [{:.0f} - {:.0f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Intervention|Number receiving intervention'] = df['formatted']

In [18]:
# we will look at the "bmi_int_dm_prev.h5" for S1
bmi_int_dm_prev = dd.read_parquet(s1_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
control_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=True, only_received=True)

# sum across replications, group, and years_after_h1yy
control_bmi_int_dm_prev_agg = control_bmi_int_dm_prev.groupby(['group', 'years_after_h1yy','replication', "time_exposure_to_risk"])['n'].sum().reset_index().compute()

# display table
control_bmi_int_dm_prev_agg

df = control_bmi_int_dm_prev_agg.groupby(['group', 'replication'])[['n']].sum().reset_index()
df['group'] = df['group'].map(group_title_dict)
df = df.groupby('group')[['n']].apply(lambda x: x.quantile([0.025,0.5,0.975])).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.0f} [{:.0f} - {:.0f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Intervention|Number receiving intervention'] = df['formatted']

In [19]:
# Now we will work on the remaining percentage columns
bmi_int_cascade = dd.read_parquet(s1_data_dir / 'bmi_int_cascade.parquet').reset_index()
# filter for only starting h1yy after 2013 and before 2017
s1_bmi_int_cascade = bmi_int_cascade.loc[(bmi_int_cascade['h1yy'] >= start_year) & (bmi_int_cascade['h1yy'] <= 2017)]

# we will look at the "bmi_int_dm_prev.h5" for S1
bmi_int_dm_prev = dd.read_parquet(s1_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
s1_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=False)

# sum across replications, group, and years_after_h1yy
s1_bmi_int_dm_prev_agg = s1_bmi_int_dm_prev.groupby(['group', 'years_after_h1yy','replication', "time_exposure_to_risk"])['n'].sum().reset_index().compute()

# display table
s1_bmi_int_dm_prev_agg

,group,years_after_h1yy,replication,time_exposure_to_risk,n
0,het_black_female,-2017,0,-2017,1302
1,het_black_female,-2017,0,0,17
2,het_black_female,-2017,0,1,14
3,het_black_female,-2017,0,2,14
4,het_black_female,-2017,0,3,22
...,...,...,...,...,...
65164,overall,22,9,-2013,576
65165,overall,22,9,22,18
65166,overall,1,0,22,2
65167,overall,3,2,22,7


## number of new dm & risk events during 7 year follow up

In [20]:
group_dm_risk_table = calc_risk_by_group(s1_bmi_int_dm_prev_agg, 7).compute()

group_dm_risk_table['group'] = group_dm_risk_table['group'].map(group_title_dict)

# New DM
df = group_dm_risk_table.groupby('group')[['dm_num']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.1f} [{:.1f} - {:.1f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Intervention|Number of new dm events during 7 year follow up'] = df['formatted']

# DM Risk
df = group_dm_risk_table.groupby('group')[['risk']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.3f} [{:.3f} - {:.3f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Intervention|7 year risk of dm'] = df['formatted']

## Number of dm per 1000 receiving the intervention

In [39]:
# we will look at the "bmi_int_dm_prev.h5" for S1
bmi_int_dm_prev = dd.read_parquet(s1_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
s1_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=True, only_received=True)

# sum across replications, group, and years_after_h1yy
s1_bmi_int_dm_prev_agg = s1_bmi_int_dm_prev.groupby(['group', 'years_after_h1yy','replication', "time_exposure_to_risk"])['n'].sum().reset_index().compute()

group_dm_risk_table = calc_risk_by_group_old(s1_bmi_int_dm_prev_agg, 7).compute()

group_dm_risk_table['group'] = group_dm_risk_table['group'].map(group_title_dict)

group_dm_risk_table['dm_per_1000'] = np.round(group_dm_risk_table['risk']*1000,0)

# New DM
df = group_dm_risk_table.groupby('group')[['dm_per_1000']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.1f} [{:.1f} - {:.1f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['Intervention|nubmer of dm event per 1000 people receiving intervention'] = df['formatted']

# Comparison

In [22]:
# we will look at the "bmi_int_dm_prev.h5" for S1
bmi_int_dm_prev_s1 = dd.read_parquet(s1_data_dir /'dm_final_output.parquet').reset_index()
bmi_int_dm_prev_s1 = bmi_int_dm_prev_s1.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# Add Overall
all_but_group = list(bmi_int_dm_prev_s1.columns[1:])
bmi_int_dm_prev_s1_overall = bmi_int_dm_prev_s1.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_s1_overall['group'] = 'overall'
bmi_int_dm_prev_s1 = dd.concat([bmi_int_dm_prev_s1, bmi_int_dm_prev_s1_overall], ignore_index=True)

# clean to control specifications
control_bmi_int_dm_prev_s1 = clean_control(bmi_int_dm_prev_s1, only_eligible=True, only_received = True)

# filter for only people eligible for intervention
bmi_int_s1_eligible_risk = calc_risk_by_group(control_bmi_int_dm_prev_s1, 7)

bmi_int_s1_eligible_risk['received_num'] = bmi_int_s1_eligible_risk['num']

In [23]:
# we will look at the "bmi_int_dm_prev.h5" for S0
bmi_int_dm_prev = dd.read_parquet(s0_data_dir /'dm_final_output.parquet').reset_index()

# Add Overall
all_but_group = list(bmi_int_dm_prev.columns[1:])
bmi_int_dm_prev_overall = bmi_int_dm_prev.groupby(all_but_group).sum().reset_index()
bmi_int_dm_prev_overall['group'] = 'overall'
bmi_int_dm_prev = dd.concat([bmi_int_dm_prev, bmi_int_dm_prev_overall], ignore_index=True)

# type the dataframe for space efficiency
bmi_int_dm_prev = bmi_int_dm_prev.astype({'group':'str', 'replication':'int16', 'bmiInt_scenario':np.int8, 'h1yy': np.int16, 'bmiInt_impacted':bool, 'dm': bool, 't_dm': np.int16, 'n': np.int16})

# clean to control specifications
control_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=True,only_received = True)

bmi_int_eligible_risk = calc_risk_by_group(control_bmi_int_dm_prev, 7)

In [24]:
num_samples = 2000

s0_sample = bmi_int_eligible_risk.groupby('group').apply(lambda x: x.sample(num_samples, replace=True)).reset_index(drop=True).compute()
s1_sample = bmi_int_s1_eligible_risk.groupby('group').apply(lambda x: x.sample(num_samples, replace=True)).reset_index(drop=True).compute()

C:\Users\jackz\AppData\Local\Temp\ipykernel_57180\1978859321.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  s0_sample = bmi_int_eligible_risk.groupby('group').apply(lambda x: x.sample(num_samples, replace=True)).reset_index(drop=True).compute()
C:\Users\jackz\AppData\Local\Temp\ipykernel_57180\1978859321.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  s1_sample = bmi_int_s1_eligible_risk.groupby('group').apply(lambda x: x.sample(num_samples, replace=True)).reset_index(drop=True).compute()


In [25]:
s0_sample = s0_sample.sort_values(by = 'group').reset_index(drop = True)
s0_sample

,group,replication,dm_num,person-time-contributed,num,risk
0,het_black_female,2,1309,53382,7626,24.521374
1,het_black_female,3,1288,52738,7534,24.422617
2,het_black_female,6,1204,51226,7318,23.503690
3,het_black_female,5,1249,49910,7130,25.025045
4,het_black_female,8,1251,53284,7612,23.477967
...,...,...,...,...,...,...
31995,overall,6,6845,655928,93704,10.435597
31996,overall,5,7146,658721,94103,10.848295
31997,overall,4,7194,659498,94214,10.908297
31998,overall,4,7194,659498,94214,10.908297


In [26]:
s1_sample = s1_sample.sort_values(by = 'group').reset_index(drop = True)
s1_sample

,group,replication,dm_num,person-time-contributed,num,risk,received_num
0,het_black_female,8,998,53284,7612,18.729825,7612
1,het_black_female,9,967,49931,7133,19.366726,7133
2,het_black_female,6,965,51226,7318,18.838090,7318
3,het_black_female,7,976,51079,7297,19.107657,7297
4,het_black_female,0,990,51807,7401,19.109387,7401
...,...,...,...,...,...,...,...
31995,overall,6,5674,655928,93704,8.650340,93704
31996,overall,5,5743,658721,94103,8.718410,94103
31997,overall,9,5915,662396,94628,8.929704,94628
31998,overall,3,5863,658539,94077,8.903041,94077


## abs change in risk

In [27]:
# absolute difference
abs_sample_diff = s1_sample[['dm_num', 'risk']] - s0_sample[['dm_num', 'risk']]
abs_sample_diff['group'] = s0_sample['group']
abs_sample_diff['received_num'] = s1_sample['received_num']

In [28]:
abs_sample_diff_plot = abs_sample_diff.copy()
abs_sample_diff_plot['group'] = abs_sample_diff_plot['group'].map(group_title_dict)

# diff_ax = sns.boxplot(x=abs_sample_diff_plot['group'],
#             y=abs_sample_diff_plot['risk'],
#             color='seagreen',
#             showfliers = False,
#             palette=palette,
#             hue=abs_sample_diff_plot['group'],
#             order=group_order,
#             hue_order=group_order)

# diff_ax.tick_params(axis='x', rotation=90)

# diff_ax.set_xlabel('')
# diff_ax.set_ylabel('Absolute risk reduction (ARR) of new DM diagnosis (intervention vs. control arm over 5-year follow up)')
# diff_ax.axhline(y=0, color='r', linestyle='-')
# diff_fig = diff_ax.get_figure()
# diff_fig.savefig("../outputs/fig3a.png")

In [29]:
# abs_sample_diff_plot.groupby('group')[['risk']].median().to_csv('../outputs/figure3a_table.csv')

df = abs_sample_diff_plot.groupby('group')[['risk']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.3f} [{:.3f} - {:.3f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['absolute change in risk'] = df['formatted']

## relative change in risk

In [30]:
# absolute difference
rel_sample_diff = (s1_sample[['risk']] - s0_sample[['risk']]) / s0_sample[['risk']]
rel_sample_diff['group'] = s0_sample['group']

In [31]:
rel_sample_diff_plot = rel_sample_diff.copy()
rel_sample_diff_plot['group'] = rel_sample_diff_plot['group'].map(group_title_dict)

# rel_ax = sns.boxplot(x=rel_sample_diff_plot['group'],
#             y=rel_sample_diff_plot['risk'],
#             color='seagreen',
#             showfliers = False,
#             palette=palette,
#             hue=rel_sample_diff_plot['group'],
#             order=group_order,
#             hue_order=group_order)

# rel_ax.tick_params(axis='x', rotation=90)

# rel_ax.set_xlabel('')
# rel_ax.set_ylabel('Relative risk reduction (RRR) of new DM diagnosis (intervention vs. control arm over 5-year follow up)')
# rel_ax.axhline(y=0, color='r', linestyle='-')
# rel_fig = rel_ax.get_figure()
# rel_fig.savefig("../outputs/fig3b.png")

# rel_sample_diff_plot.groupby('group')[['risk']].median().to_csv('../outputs/figure3b_table.csv')

df = rel_sample_diff_plot.groupby('group')[['risk']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.3f} [{:.3f} - {:.3f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['relative change in risk'] = df['formatted']

## Number dm cases averted per 1000 & NNT

In [32]:
abs_sample_diff_plot['dm_per_1000'] = abs_sample_diff_plot['risk'] * (-1000)
abs_sample_diff_plot['NNT'] = -np.round(abs_sample_diff_plot['received_num']/abs_sample_diff_plot['dm_num'], 0)

# dm_per_1000_ax = sns.boxplot(x=abs_sample_diff_plot['group'],
#             y=abs_sample_diff_plot['NNT'],
#             color='seagreen',
#             showfliers = False,
#             palette=palette,
#             hue=abs_sample_diff_plot['group'],
#             order=group_order,
#             hue_order=group_order)

# dm_per_1000_ax.tick_params(axis='x', rotation=90)

# dm_per_1000_ax.set_xlabel('')
# dm_per_1000_ax.set_ylabel('Number needed to treat (NNT) to avert one new DM diagnosis (intervention vs. control arm over 5-year follow up)')
# dm_per_1000_ax.axhline(y=0, color='r', linestyle='-')
# dm_per_1000_fig = dm_per_1000_ax.get_figure()
# dm_per_1000_fig.savefig("../outputs/fig3c.png")

# dm cases averted per 1000
# df = abs_sample_diff_plot.groupby('group')[['dm_per_1000']].quantile([0.025,0.5,0.975]).unstack().reset_index()
# df.columns = ['group',0.025, 0.5, 0.975]
# df['formatted'] = df.apply(
#     lambda row: '{:.0f} [{:.0f} - {:.0f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
# )
# df = rearrange_group_order(df)
# SA_summary_df['num dm cases averted per 1000'] = df['formatted']

# NNT
df = abs_sample_diff_plot.groupby('group')[['NNT']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.0f} [{:.0f} - {:.0f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)
SA_summary_df['NNT'] = df['formatted']

## total number of dm averted

In [33]:
abs_sample_diff_plot['dm_num_prevented'] = abs_sample_diff_plot['dm_num'] * -1
# dm_prevented_ax = sns.boxplot(x=abs_sample_diff_plot['group'],
#             y=abs_sample_diff_plot['dm_num_prevented'],
#             color='seagreen',
#             showfliers = False,
#             palette=palette,
#             hue=abs_sample_diff_plot['group'],
#             order=group_order,
#             hue_order=group_order)

# dm_prevented_ax.tick_params(axis='x', rotation=90)

# dm_prevented_ax.set_xlabel('')
# dm_prevented_ax.set_ylabel('Number of DM cases averted(intervention vs. control arm (over 5-year follow up)')
# dm_prevented_ax.axhline(y=0, color='r', linestyle='-')
# dm_prevented_fig = dm_prevented_ax.get_figure()
# dm_prevented_fig.savefig("../outputs/fig3d.png")

# abs_sample_diff_plot.groupby('group')[['dm_num_prevented']].median().to_csv('../outputs/figure3d_table.csv')

df = abs_sample_diff_plot.groupby('group')[['dm_num_prevented']].quantile([0.025,0.5,0.975]).unstack().reset_index()
df.columns = ['group',0.025, 0.5, 0.975]
df['formatted'] = df.apply(
    lambda row: '{:.0f} [{:.0f} - {:.0f}]'.format(row[0.50], row[0.025], row[0.975]), axis=1
)
df = rearrange_group_order(df)

SA_summary_df['total number of dm averted'] = df['formatted']

# Save summary df

In [40]:
SA_summary_df

,group,Control|Number receiving intervention,Control|Number of new dm events during 7 year follow up,Control|7 year risk of dm,Control|nubmer of dm event per 1000 people receiving intervention,Intervention|Number receiving intervention,Intervention|Number of new dm events during 7 year follow up,Intervention|7 year risk of dm,Intervention|nubmer of dm event per 1000 people receiving intervention,absolute change in risk,relative change in risk,num dm cases averted per 1000,NNT,total number of dm averted
0,Black HET Women,7360 [7131 - 7623],3633.0 [3432.7 - 3733.1],26.521 [25.834 - 26.893],171.0 [163.2 - 176.6],7360 [7131 - 7623],3376.5 [3198.0 - 3459.5],24.490 [24.143 - 24.856],134.0 [131.0 - 136.8],-5.155 [-6.584 - -3.865],-0.210 [-0.260 - -0.166],5155 [3865 - 6584],27 [21 - 47],271 [162 - 342]
1,White HET Women,1908 [1767 - 1950],699.0 [687.7 - 742.3],23.240 [22.408 - 24.271],153.5 [146.2 - 169.0],1908 [1767 - 1950],656.0 [627.2 - 698.6],21.547 [20.972 - 22.794],129.0 [119.3 - 140.2],-3.599 [-6.387 - -0.763],-0.163 [-0.267 - -0.036],3599 [763 - 6387],38 [20 - 171],49 [11 - 89]
2,Hispanic HET Women,2084 [2021 - 2197],810.5 [760.8 - 861.1],23.178 [22.280 - 23.778],152.0 [144.2 - 159.3],2084 [2021 - 2197],755.0 [713.7 - 807.9],21.713 [20.861 - 22.487],124.5 [118.5 - 135.1],-3.641 [-5.644 - -1.365],-0.172 [-0.247 - -0.065],3641 [1365 - 5644],37 [22 - 208],57 [10 - 92]
3,Black HET Men,4719 [4612 - 4840],1212.0 [1150.8 - 1252.2],18.768 [17.748 - 19.040],128.5 [113.5 - 132.8],4719 [4612 - 4840],1078.5 [1022.0 - 1122.2],16.601 [15.948 - 17.004],98.5 [94.5 - 104.0],-4.006 [-5.367 - -1.537],-0.218 [-0.285 - -0.093],4006 [1537 - 5367],36 [26 - 112],133 [43 - 182]
4,White HET Men,1128 [1066 - 1144],255.5 [230.0 - 279.6],17.843 [16.019 - 19.001],119.0 [98.4 - 125.3],1128 [1066 - 1144],219.5 [185.5 - 247.5],15.232 [13.233 - 16.752],85.0 [77.2 - 97.8],-4.198 [-6.555 - -0.263],-0.250 [-0.373 - -0.019],4198 [263 - 6555],33 [-166 - 142],32 [0 - 55]
5,Hispanic HET Men,1568 [1482 - 1670],357.0 [341.4 - 390.9],18.021 [17.093 - 18.684],126.0 [119.2 - 135.6],1568 [1482 - 1670],296.0 [273.8 - 331.6],15.044 [13.747 - 15.782],91.0 [85.5 - 96.0],-5.032 [-6.973 - -3.320],-0.279 [-0.360 - -0.194],5032 [3320 - 6973],28 [17 - 67],55 [24 - 89]
6,Black WWID,838 [768 - 909],384.0 [348.8 - 430.9],27.922 [26.161 - 30.059],195.5 [163.1 - 207.3],838 [768 - 909],363.5 [318.7 - 386.4],25.700 [23.881 - 27.862],156.0 [144.5 - 176.7],-4.470 [-8.238 - 1.669],-0.164 [-0.289 - 0.073],4470 [-1669 - 8238],27 [-130 - 418],25 [-17 - 66]
7,White WWID,1051 [942 - 1155],412.0 [383.4 - 487.2],29.902 [27.644 - 30.870],215.0 [188.2 - 231.2],1051 [942 - 1155],374.5 [342.6 - 443.0],26.477 [24.798 - 28.160],165.5 [148.1 - 190.8],-6.699 [-11.267 - -0.774],-0.215 [-0.353 - -0.029],6699 [774 - 11267],21 [-226 - 152],45 [-12 - 103]
8,Hispanic WWID,448 [412 - 491],161.5 [146.2 - 202.4],27.338 [23.927 - 31.018],184.5 [168.0 - 232.4],448 [412 - 491],146.5 [120.5 - 183.3],24.440 [19.764 - 27.884],155.5 [117.1 - 190.4],-5.274 [-14.839 - 3.917],-0.194 [-0.472 - 0.163],5274 [-3917 - 14839],16 [-240 - 148],18 [-22 - 57]
9,Black MWID,2090 [1907 - 2289],390.0 [372.8 - 454.6],14.048 [13.662 - 14.823],106.0 [98.1 - 113.3],2090 [1907 - 2289],333.0 [300.9 - 396.8],11.957 [11.076 - 13.003],77.5 [69.2 - 83.8],-4.267 [-5.997 - -2.464],-0.282 [-0.375 - -0.177],4267 [2464 - 5997],34 [18 - 128],57 [18 - 118]


In [35]:
SA_summary_df.to_csv(f'../results/SA/{config_type}_cov_{coverage_rate}_eff_{effectiveness}.csv', index = False)